In [ ]:
import dibs_notebook_header
import dibs
import itertools
import multiprocessing
import numpy as np
import os
import pandas as pd
import time

In [ ]:
num_procs = 8

In [ ]:
def avg(low,high):
    return (low+high)/2

In [ ]:
plow, phigh  = 30, 300
pmid = avg(plow,phigh)
lrlow, lrhigh  = 50, 300
lrmid = avg(lrlow, lrhigh)
eelow, eehigh = 150, 500
eemid=avg(eelow,eehigh)
kwargs_product = [{'tsne_perplexity': p,
                   'tsne_early_exaggeration': ee,
                   'tsne_learning_rate': lr,
                   'tsne_n_components': 2, 
                   'tsne_n_jobs': 1,
                   'gmm_n_components': 8,  
                   'cross_validation_n_jobs': 1,
                  } for p,lr, ee in itertools.product([plow,pmid,phigh],[lrlow,lrhigh],[eelow,eemid,eehigh])]
names = [f'Pipeline_{i}' for i in range(len(kwargs_product))]
print('Number of combos:', len(kwargs_product))
# kwargs_product[:2]

In [ ]:
len(list(zip(names, kwargs_product)))

In [ ]:
all_files = [os.path.join(dibs.config.DEFAULT_TRAIN_DATA_DIR, file) for file in os.listdir(dibs.config.DEFAULT_TRAIN_DATA_DIR)]
train_data = half_files = all_files[:int(len(all_files) * 0.35)]
train_data[:3]

In [ ]:
pipe_results = [dibs.pipeline.PipelineMimic(name, **kwargs).add_train_data_source(*train_data) for name, kwargs in zip(names, kwargs_product)]
# print("Number of possible results", len(pipe_results))

In [ ]:
len(pipe_results)

In [ ]:
start = time.perf_counter()
with multiprocessing.Pool(num_procs) as pool:
    pipe_results = [pool.apply_async(pipe.build) for pipe in pipe_results]
    pipe_results = [res.get() for res in pipe_results]
end = time.perf_counter()
print('Total compute time:', end-start, 'seconds.')

In [ ]:
for i, pipe in enumerate(pipe_results):
    p,lr,ee = pipe.tsne_perplexity, pipe.tsne_learning_rate, pipe.tsne_early_exaggeration
    print('perp', p , 'ee', ee, 'lr', lr)
    pipe.plot_clusters_by_assignments(show_now=True, save_to_file=True, figsize=(20,15),s=1.5)
    print('-----------------------------------')

In [ ]:
print('All done!')

In [ ]:
dibs.pipeline.logger.info('All done!')